In [2]:
import numpy as np 
import pandas as pd 

import nltk
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag, ne_chunk
from nltk.tree import Tree

import inspect

import textacy.vsm
from textacy.vsm import Vectorizer

import scipy.sparse as sp

from tqdm import *
import re

In [3]:
tweets = pd.read_csv("unique.txt", sep="\r\n", names=['tweet_texts'], encoding = 'utf-8', engine='python')
tweets.dropna()
tweets.drop(tweets.index[0], inplace=True)
tweets.tweet_texts[1]

'9) uae may purchase our astra which is a 70-km range air-to-air missile. jordan is interested in buying the kamov 226 t multi-utility helicopters to be built by india-russia. it may also buy other hardware developed by the drdo.'

In [4]:
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub('https\S+', '', x))   
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub('http\S+', '', x))
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub('(\s)@\w+', '', x))

tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub('#', '', x))
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub('|','',x))

tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: x.replace('RT', ''))

emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub(emoji_pattern,'',x))

temp = re.compile(u'\u0900-\u097F', flags = re.UNICODE)

tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: temp.sub('',x))
# tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: x.replace('\d+(.*?)[\u0900-\u097F]', ''))

with open("cleaned_sensitive.txt", "w") as f:
		for tweet in tweets.tweet_texts:
			f.write(tweet+'\n')

In [6]:
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)
nltk_tweets = []

for tweet in tweets.tweet_texts:
    tokenized = tokenizer.tokenize(tweet)
    if(len(tokenized) >= 2):
        nltk_tweets.append(tokenized)

nltk_pos = []

for tweet in nltk_tweets:
    nltk_pos.append(pos_tag(tweet))

print(nltk_pos)

[[(u'9', 'CD'), (u')', ')'), (u'uae', 'NN'), (u'may', 'MD'), (u'purchase', 'VB'), (u'our', 'PRP$'), (u'astra', 'NN'), (u'which', 'WDT'), (u'is', 'VBZ'), (u'a', 'DT'), (u'70', 'CD'), (u'-', ':'), (u'km', 'NN'), (u'range', 'NN'), (u'air-to-air', 'JJ'), (u'missile', 'NN'), (u'.', '.'), (u'jordan', 'NN'), (u'is', 'VBZ'), (u'interested', 'JJ'), (u'in', 'IN'), (u'buying', 'VBG'), (u'the', 'DT'), (u'kamov', 'NN'), (u'226', 'CD'), (u't', 'JJ'), (u'multi-utility', 'NN'), (u'helicopters', 'NNS'), (u'to', 'TO'), (u'be', 'VB'), (u'built', 'VBN'), (u'by', 'IN'), (u'india-russia', 'NN'), (u'.', '.'), (u'it', 'PRP'), (u'may', 'MD'), (u'also', 'RB'), (u'buy', 'VB'), (u'other', 'JJ'), (u'hardware', 'NN'), (u'developed', 'VBN'), (u'by', 'IN'), (u'the', 'DT'), (u'drdo', 'NN'), (u'.', '.')], [(u'(', '('), (u'drdo', 'NN'), (u')', ')'), (u'today', 'NN'), (u'test-fired', 'VBD'), (u'the', 'DT'), (u'surface', 'NN'), (u'to', 'TO'), (u'air', 'NN'), (u'missile', 'NN'), (u'(', '('), (u'sam', 'NN'), (u')', ')'), (u

In [72]:
# from run_ner import TwitterNER

In [73]:
# ner = TwitterNER()

In [7]:
nltk_ents = []

# for tweet in tqdm(nltk_tweets):
#     entity_tagged_tweet = ner.get_entities(tweet)
#     nltk_ents.append([ (tweet[tag[0]:tag[1]][0] , tag[2]) for tag in entity_tagged_tweet])


for tweet in tqdm(nltk_pos):
    nouns = ['NN','NNP','NNS','NNPS']
    # entity_tagged_tweet = ner.get_entities(tweet)
    '''
        Find a better way to find content words in a tweet.
    '''
    # sensitive_ents.append([ (tweet[tag[0]:tag[1]][0] , tag[2]) for tag in entity_tagged_tweet])
    nltk_ents.append([ (word[0],word[1])  for word in tweet if word[1] in nouns])

content_tweets = []
for pos_tweet, content_words in tqdm(zip(nltk_pos, nltk_ents)):
    # we'll start by definitely appending all of the entities
    single_tweet_content = [word[0] for word in content_words]
    # next, add the token if it is a number
    for token in pos_tweet: 
        if token[1] == 'CD': # CD = cardinal number
            single_tweet_content.append(token[0])
    content_tweets.append(single_tweet_content)

100%|██████████| 960/960 [00:00<00:00, 98897.97it/s]


In [75]:
# nltk_ents


In [8]:
content_tweets = []
for pos_tweet, content_words in tqdm(zip(nltk_pos, nltk_ents)):
    # we'll start by definitely appending all of the entities
    single_tweet_content = [word[0] for word in content_words]
    
    # next, add the token if it is a number
    for token in pos_tweet: 
        if token[1] == u'CD': # CD = cardinal number
            single_tweet_content.append(token[0])
    content_tweets.append(single_tweet_content)
    
# content_tweets = nltk_tweets

100%|██████████| 960/960 [00:00<00:00, 53117.67it/s]


In [9]:
tweet_num = 11
print ("original_tweet \n" + str(nltk_tweets[tweet_num]) 
       + "\n\ncontent_tweet\n" + str(content_tweets[tweet_num])
      )

original_tweet 
[u'drdo', u'successfully', u'test', u'fired', u'quick', u'reaction', u'surface', u'to', u'air', u'missile', u'(', u'qrsam', u')', u'from', u'odisha', u'defence', u'base']

content_tweet
[u'drdo', u'reaction', u'surface', u'air', u'missile', u'qrsam', u'defence', u'base']


In [10]:
vectorizer = Vectorizer(tf_type='linear', apply_idf=True, idf_type='smooth')

In [11]:
term_matrix = vectorizer.fit_transform(nltk_tweets)
np_matrix = term_matrix.todense()

In [12]:
len(vectorizer.vocabulary_terms)

2270

In [81]:
# for key in sorted(vectorizer.vocabulary_terms)[700:715]:
#     print(key, vectorizer.vocabulary_terms[key])

In [82]:
# for token in content_tweets[500]:
#     print (token, vectorizer.vocabulary_terms[token[0]], np.max(np_matrix[:,vectorizer.vocabulary_terms[token[0]]]))

In [13]:
tfidf_dict = {}
content_vocab = []
for tweet in content_tweets: 
    for token in tweet: 
        if token not in tfidf_dict.keys(): 
#             print type(vectorizer.vocabulary_terms)
            if token in vectorizer.vocabulary_terms.keys():
                content_vocab.append(token)
                tfidf_dict[token] = np.max(np_matrix[:,vectorizer.vocabulary_terms[token]])

In [14]:
len(tfidf_dict)

1438

In [61]:
from pymprog import *
begin('COWTS')

model('COWTS') is the default model.

In [62]:
# Defining my first variable, x 
# This defines whether or not a tweet is selected
x = var('x', len(nltk_tweets), bool)

# Check this worked

In [63]:
# Also defining the second variable, which defines
# whether or not a content word is chosen
y = var('y', len(content_vocab), bool)

In [64]:
len(y), y[0]

(1438, 0 <= y[0] <= 1 binary)

In [65]:
maximize(sum(x) + sum([tfidf_dict[content_vocab[j]]*y[j] for j in range(len(y))]));

In [66]:
## Maximum length of the entire tweet summary

# Was 150 for the tweet summary, 
# But generated a 1000 word summary for CONABS
L = 500

# hiding the output of this line since its a very long sum 
sum([x[i]*len(nltk_tweets[i]) for i in range(len(x))]) <= L;

In [67]:
def content_words(i):
    '''Given a tweet index i (for x[i]), this method will return the indices of the words in the 
    content_vocab[] array
    Note: these indices are the same as for the y variable
    '''
    tweet = nltk_tweets[i]
    content_indices = []
    
    for token in tweet:
        if token in content_vocab:
            content_indices.append(content_vocab.index(token))
    return content_indices

In [68]:
def tweets_with_content_words(j):
    '''Given the index j of some content word (for content_vocab[j] or y[j])
    this method will return the indices of all tweets which contain this content word
    '''
    content_word = content_vocab[j]
    
    index_in_term_matrix = vectorizer.vocabulary_terms[content_word]
    
    matrix_column = np_matrix[:, index_in_term_matrix]
    
    return np.nonzero(matrix_column)[0]

In [69]:
for j in range(len(y)):
    sum([x[i] for i in tweets_with_content_words(j)])>= y[j]

In [70]:
for i in range(len(x)):
    sum(y[j] for j in content_words(i)) >= len(content_words(i))*x[i]

In [71]:
solve()

(0,
 'The MIP problem instance has been successfully solved. (This code\ndoes {\\it not} necessarily mean that the solver has found optimal\nsolution. It only means that the solution process was successful.)')

In [72]:
result_x =  [value.primal for value in x]
result_y = [value.primal for value in y]

In [73]:
end()

model('COWTS') is not the default model.

In [74]:
chosen_tweets = np.nonzero(result_x)
chosen_words = np.nonzero(result_y)

In [75]:
len(chosen_tweets[0]), len(chosen_words[0])

(54, 307)

In [78]:
cowts_tweets = []
for i in chosen_tweets[0]:
    cowts_tweets.append(" ".join(nltk_tweets[i]))
cowts_tweets

[u'dual colour missile approach warning system for fighter aircraft',
 u'\u0938 \u0947 \u0928 \u093f \u0935 \u0947 \u0926\u0928',
 u'\u092d \u093e \u0930\u0924 \u0915 \u094b new ballistic missile \u0926 \u0947 \u0902',
 u'\u0936 \u0940 \u0918 \u094d \u0930 \u092e \u0941 \u0939 \u0948 \u092f \u093e \u0915\u0930\u0935 \u093e \u090f \u0924 \u094b \u092c \u0947 \u0939\u0924\u0930 \u0939 \u094b \u0917 \u093e',
 u'hindistan ile rusya daha \xf6nce s\u0131v\u0131 yak\u0131tl\u0131 ramjetli brahmos f\xfczesini geli\u015ftirmi\u015flerdi . \u015fimdi iki \xfclke kat\u0131 yak\u0131tl\u0131 ramjetli astra - 2 adl\u0131 hava-hava f\xfczesi geli\u015ftiriyorlar .',
 u'drgsatheeshreddychairmandrdosecretaryddr & ddrschristopher',
 u'it media cell',
 u'drmaahi 2799',
 u'agni 5',
 u'drdo maleinindia',
 u'day hit :',
 u'- 4900km in 19 mins \ud83d \ude0a \ud83d \ude0a',
 u'jai hind',
 u'indvspak u19cwc',
 u'\u0aad \u0abe \u0ab0\u0aa4 \u0ac0 \u0aaf \u0ab8 \u0ac7 \u0aa8 \u0abe \u0aae \u0abe \u0a82 \u0aec\u

In [79]:
cowts_dataframe = pd.DataFrame(cowts_tweets)

In [80]:
cowts_dataframe.to_pickle('cowts_tweets-1.pkl')

In [81]:
cowts_tweets = pd.read_pickle('cowts_tweets-1.pkl')

In [82]:
tweets = []
for tweet in cowts_tweets[0]:
    tweets.append(tokenizer.tokenize(tweet))

In [83]:
tweets = [tweet for tweet in tweets if len(tweet) > 1]
len(tweets)

54

In [84]:
from nltk.util import bigrams
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cosine
import kenlm
import math

In [85]:
(list(bigrams(tweets[0])))

[(u'dual', u'colour'),
 (u'colour', u'missile'),
 (u'missile', u'approach'),
 (u'approach', u'warning'),
 (u'warning', u'system'),
 (u'system', u'for'),
 (u'for', u'fighter'),
 (u'fighter', u'aircraft')]

In [86]:
all_bigrams = [list(bigrams([token for token in tweet_list])) for tweet_list in tweets]

In [87]:
starting_nodes = [single_bigram[0] for single_bigram in all_bigrams]

In [88]:
end_nodes = [single_bigram[-1] for single_bigram in all_bigrams]

In [89]:
all_bigrams = [node for single_bigram in all_bigrams for node in single_bigram]

In [90]:
all_bigrams = list(set(all_bigrams))

In [91]:
def make_bigram_graph(all_bigrams, start_node):
    bigram = all_bigrams[:]
    
    '''
    Given a bigram, with a defined start node and defined end nodes, this method
    returns a dictionary which serves as a graph for that bigram 
    '''
    def find_children(bigram, node):
        '''
        Given a node, this method finds all its children 
        '''
        second_word = node[1]
        
        children = [node for node in bigram if node[0] == second_word]
        
        return children
   
    bigram_graph = {}
    # start by adding the start node
    bigram_graph[start_node] = find_children(all_bigrams, start_node)
    bigram.remove(start_node)
    
    nodes_to_check = []
    for i in find_children(bigram, start_node):
        nodes_to_check.append(i)
        
    while nodes_to_check: 
        node = nodes_to_check.pop()
        if node in bigram: 
            bigram_graph[node] = find_children(bigram, node)
            bigram.remove(node)
            for i in find_children(bigram, node):
                nodes_to_check.append(i)
    return bigram_graph

In [92]:
bigram_graph = make_bigram_graph(all_bigrams, starting_nodes[1])

In [93]:
len(bigram_graph)

235

In [94]:
def breadth_first_search(bigram_graph, start_node, end_node):
    '''
    This method takes as input a graph, a start node and an end node
    and returns all paths which have a length between 10 and 16
    between the two nodes.
    '''
    graph_to_manipulate = dict(bigram_graph)
    
    queue = []
    paths_to_return = []
    queue.append([start_node])
    
    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        # get the last node from the path
        node = path[-1]
        # path found
        if node == end_node:
            if (len(path) < 16) and (len(path) > 10): #limit path length 
                paths_to_return.append(path)
        # enumerate all adjacent nodes, construct a new path and push it into the queue
        for adjacent in graph_to_manipulate.get(node, []):
            new_path = list(path)
            new_path.append(adjacent)
            queue.append(new_path)
        if node in graph_to_manipulate: 
            del graph_to_manipulate[node] # prevents circular references

    return paths_to_return

In [95]:
path = breadth_first_search(bigram_graph, starting_nodes[1], end_nodes[2])

In [96]:
bigram_paths = []

for single_start_node in tqdm(starting_nodes): 
    bigram_graph = make_bigram_graph(all_bigrams, single_start_node)
    for single_end_node in end_nodes:
        possible_paths = breadth_first_search(bigram_graph, single_start_node, single_end_node)
        for path in possible_paths: 
            bigram_paths.append(path)

100%|██████████| 54/54 [00:01<00:00, 31.34it/s]


In [97]:
len(bigram_paths)

302

In [98]:
for tweet in tweets:
    bigram_paths.append(list(bigrams([token for token in tweet])))

In [99]:
def make_list(bigram_path):
    '''
    This method takes a bigram path (eg. [(u'hello', u'world'), (u'world', u'!')]) and returns 
    a list of unicode (eg [u'hello', u'world', u'!')
    '''
    unicode_list = []
    unicode_list.append(bigram_path[0][0])
    unicode_list.append(bigram_path[0][1])
    
    for bigram in bigram_path[1:]:
        unicode_list.append(bigram[1])
    
    return unicode_list

In [100]:
word_paths = []
for path in tqdm(bigram_paths): 
    word_paths.append(make_list(path))

100%|██████████| 356/356 [00:00<00:00, 41691.25it/s]


In [101]:
np.save('term_matrix-1.npy', np_matrix)

In [102]:
vocab_to_idx = vectorizer.vocabulary_terms
term_matrix = np.load('term_matrix-1.npy');
print type(vocab_to_idx)
print type(term_matrix)
print len(term_matrix)

<type 'dict'>
<type 'numpy.ndarray'>
960


In [103]:
def informativeness(word_path):
    '''
    This method returns the cosine difference between
    a tweet path and the mean of the tf-idf term matrix
    
    Input = word path (as a unicode list)
    Ouptut = cosine difference (scalar value)
    '''
    tfidf_mean = np.mean(term_matrix, axis = 0)
    
    # First, I need to construct the tf-idf vector
    tfidf_path = np.zeros(len(tfidf_mean))
    
    for word in word_path: 
        word_idx = vocab_to_idx[word]
        tfidf_path[word_idx] = np.max(term_matrix[:,word_idx])
   
    cosine_difference = cosine(tfidf_mean, tfidf_path)
    return cosine_difference

In [104]:
informativeness(word_paths[1])

0.8799384987741607

In [105]:
kenlm_model = kenlm.Model('glob.arpa')

In [111]:
L = 300

In [112]:
def linguistic_quality(word_path):
    '''
    This method takes a word path, and returns a linguistic quality score 
    '''
#     print word_path[]
#     for token in word_path:
#         print type(token)
    path_string = str(" ").join([token.encode('ascii', 'ignore') for token in word_path])
    
    ll_score = math.log(10**kenlm_model.score(path_string, bos = True, eos = True), 2)/L
    
    # scale by 100 so this meaningfully impacts the Integer Programming Solution
    return 1/(1-ll_score)

In [113]:
begin('COWABS')

model('COWABS') is the default model.

In [114]:
x = var(str('x'), len(word_paths), bool)

In [115]:
y = var(str('y'), len(content_vocab), bool)

In [116]:
maximize(sum([linguistic_quality(word_paths[i])*informativeness(word_paths[i])*x[i] for i in range(len(x))]) + 
         sum(y));

In [117]:
sum([x[i]*len(word_paths[i]) for i in range(len(x))]) <= L;

In [118]:
def content_words(i):
    '''Given a word path index i (for x[i]), this method will return the indices of the words in the 
    content_vocab[] array
    Note: these indices are the same as for the y variable
    '''
    path = word_paths[i]
    content_indices = []
    
    for word in path:
        if word in content_vocab:
            content_indices.append(content_vocab.index(word))
    return content_indices

In [119]:
def paths_with_content_words(j):
    '''Given the index j of some content word (for content_vocab[j] or y[j])
    this method will return the indices of all tweets which contain this content word
    '''
    content_word = content_vocab[j]
    
    indices = []
    
    for i in range(len(word_paths)):
        if content_word in word_paths[i]:
            indices.append(i)
    
    return indices

In [120]:
for j in range(len(y)):
    sum([x[i] for i in paths_with_content_words(j)])>= y[j]

In [121]:
for i in range(len(x)):
    sum(y[j] for j in content_words(i)) >= len(content_words(i))*x[i]

In [122]:
solve()

(0,
 'The MIP problem instance has been successfully solved. (This code\ndoes {\\it not} necessarily mean that the solver has found optimal\nsolution. It only means that the solution process was successful.)')

In [123]:
result_x =  [value.primal for value in x]
result_y = [value.primal for value in y]

In [124]:
end()

model('COWABS') is not the default model.

In [125]:
chosen_paths = np.nonzero(result_x)
chosen_words = np.nonzero(result_y)

In [126]:
for i in chosen_paths[0]:
    print "-------"
    print str(" ").join([token.encode('ascii', 'ignore') for token in word_paths[i]])

-------
hindistan ile rusya daha nce sv yaktl ramjetli brahmos fzesini gelitirmilerdi . txt
-------
rafael's indian army for d a5 mission      
-------
congratulation t strategic command f indian missile k - details features specification currentaffairs gk
-------
wikileaksindia wikileaks drdo prithvi-ii missile - ii      
-------
aur bilawal missile -     drdo ) proposalsbyindustriesforantitankmissiletechnology whatarenagandnamica whatisantitankmissiletechnology
-------
congrats d armed forces & defence industry for testing agni 1missile n test fr hr range
-------
twitter _15_clean_data . hstdv hypersonic cruise missile pritv-vi jaisa hal hone ki
-------
dual colour missile approach warning system for fighter aircraft
-------
      
-------
                      
-------
drgsatheeshreddychairmandrdosecretaryddr & ddrschristopher
-------
it media cell
-------
drmaahi 2799
-------
agni 5
-------
drdo maleinindia
-------
day hit :
-------
jai hind
-------
indvspak u19cwc
-------
the atom

In [127]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [128]:
references_bleu = []
ref = []
for tweet in nltk_tweets:
    ref.append([token.encode('ascii', 'ignore').lower() for token in tweet])
references_bleu.append(ref)

In [129]:
candidates_bleu = []
can = []
for i in chosen_paths[0]:
    can.append([token.encode('ascii', 'ignore') for token in word_paths[i]])

candidates_bleu = [token for tweet in can for token in tweet]
candidates_bleu = [candidates_bleu]

In [130]:
score = corpus_bleu(references_bleu, candidates_bleu)
print("Bleu Score: ", score)

('Bleu Score: ', 0.7329994537163145)


In [131]:
punct = [".",""]
reference_rouge = ""
for ref in references_bleu[0]:
    reference_rouge += (" ".join(ref))

# print reference_rouge
candidate_rouge = ""
for can in candidates_bleu:
    candidate_rouge += ((" ".join([token for token in can if token not in punct])) + " ")
# print candidate_rouge

In [132]:
from rouge import Rouge
r = Rouge()
r.get_scores(reference_rouge, candidate_rouge)

[{'rouge-1': {'f': 0.11262939834865768, 'p': 0.06031042128603104, 'r': 0.85},
  'rouge-2': {'f': 0.030204322839515418,
   'p': 0.0155203895313451,
   'r': 0.5604395604395604},
  'rouge-l': {'f': 0.05836474154064016,
   'p': 0.05809312638580931,
   'r': 0.81875}}]